In [2]:
# imports
import pandas as pd
# from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.dummy import DummyClassifier
import statsmodels.api as sm


In [3]:
X_train = pd.read_csv('data/Training_set_values.csv', index_col='id')
X_test = pd.read_csv('data/Test_set_values.csv', index_col='id')
y_train = pd.read_csv('data/Training_set_labels.csv', index_col='id')

In [4]:
X_train.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [5]:
y_train.head()

,status_group
id,
69572,functional
8776,functional
34310,functional
67743,non functional
19728,functional


## Merging the X and y train data.

In [6]:
df = X_train.merge(y_train, on='id')

We merged the X train and y train data to build a dataframe that we could use to testing our different models on. The data set from where the data came did not have a y test dataset. We will split the merged dataset and create a hold out group later.

In [7]:
df.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,
69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional


# Data Cleaning

In [8]:
df.isna().sum()

amount_tsh                   0
date_recorded                0
funder                    3635
gps_height                   0
installer                 3655
longitude                    0
latitude                     0
wpt_name                     0
num_private                  0
basin                        0
subvillage                 371
region                       0
region_code                  0
district_code                0
lga                          0
ward                         0
population                   0
public_meeting            3334
recorded_by                  0
scheme_management         3877
scheme_name              28166
permit                    3056
construction_year            0
extraction_type              0
extraction_type_group        0
extraction_type_class        0
management                   0
management_group             0
payment                      0
payment_type                 0
water_quality                0
quality_group                0
quantity

In [9]:
df = df.drop(['scheme_name', 'date_recorded', 'wpt_name', 'subvillage', 'lga', 'ward', 'recorded_by', 'quantity_group', 'quantity_group'], axis = 1)
df = df.fillna('missing', axis = 1)

We are dropping scheme name, and filling the NaN values with a string `missing`. We dropped scheme name since 28,000 values are missing. We replaced NaN values with `missing` to keep the rows in our dataframe. We will onehotencode the data frame later.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59400 entries, 69572 to 26348
Data columns (total 32 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   amount_tsh             59400 non-null  float64
 1   funder                 59400 non-null  object 
 2   gps_height             59400 non-null  int64  
 3   installer              59400 non-null  object 
 4   longitude              59400 non-null  float64
 5   latitude               59400 non-null  float64
 6   num_private            59400 non-null  int64  
 7   basin                  59400 non-null  object 
 8   region                 59400 non-null  object 
 9   region_code            59400 non-null  int64  
 10  district_code          59400 non-null  int64  
 11  population             59400 non-null  int64  
 12  public_meeting         59400 non-null  object 
 13  scheme_management      59400 non-null  object 
 14  permit                 59400 non-null  object 
 15

We are creating a hold out data set which we will test our final model on.

In [11]:
df, holdout = train_test_split(df, test_size = .1)

In [12]:
X = df.drop('status_group', axis=1)
y = df['status_group']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

In [13]:
X_train.head()

,amount_tsh,funder,gps_height,installer,longitude,latitude,num_private,basin,region,region_code,...,payment,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group
id,,,,,,,,,,,,,,,,,,,,,
15720,0.0,Tasaf,0,GEOTAN,34.096245,-3.731346,0,Internal,Shinyanga,17,...,pay per bucket,per bucket,soft,good,enough,shallow well,shallow well,groundwater,communal standpipe multiple,communal standpipe
58308,0.0,missing,0,missing,33.643917,-9.295407,0,Lake Nyasa,Mbeya,12,...,never pay,never pay,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe
26036,20.0,Water User As,248,Wa,38.386402,-6.335334,0,Wami / Ruvu,Pwani,6,...,pay per bucket,per bucket,soft,good,enough,river,river/lake,surface,communal standpipe,communal standpipe
25508,50.0,Unicef,-23,Unisef,40.109778,-10.231825,0,Ruvuma / Southern Coast,Mtwara,99,...,pay per bucket,per bucket,soft,good,enough,spring,spring,groundwater,communal standpipe multiple,communal standpipe
1534,0.0,Rudep,1668,DWE,32.095697,-8.990321,0,Lake Rukwa,Rukwa,15,...,never pay,never pay,soft,good,enough,machine dbh,borehole,groundwater,hand pump,hand pump


In [14]:
#X_train.to_csv('X_train.csv')
#X_test.to_csv('X_test.csv')
#y_train.to_csv('y_train.csv')
#y_test.to_csv('y_test.csv')
#
#df.to_csv('holdout_data.csv')

# Baseline Dummy Model

In [15]:
dummy_model = DummyClassifier(strategy='most_frequent', random_state=42)
dummy_model.fit(X_train, y_train)

DummyClassifier(random_state=42, strategy='most_frequent')

In [16]:
print('Accuracy Score Train:', dummy_model.score(X_train, y_train))
print('Accuracy Score Test:', dummy_model.score(X_test, y_test))

Accuracy Score Train: 0.5425614166354907
Accuracy Score Test: 0.5426112981668537


In [17]:
log_loss_dummy = cross_val_score(dummy_model, X_train, y_train, scoring='neg_log_loss')
log_loss_dummy = -log_loss_dummy.mean()
print('Log Loss:', log_loss_dummy)

Log Loss: 15.799368945231496


# Logistic Regression Model -Sanjit

Importing relevant tools

In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score

import seaborn as sns

In [19]:
logreg = LogisticRegression()
ss = StandardScaler()

X_train_num = X_train.select_dtypes(exclude= object)
X_test_num = X_test.select_dtypes(exclude= object)

X_train_num_scaled = ss.fit_transform(X_train_num)
X_test_num_scaled = ss.transform(X_test_num)

logreg.fit(X_train_num_scaled, y_train)

preds = logreg.predict(X_test_num_scaled)

In [20]:
test_accuracy = accuracy_score(y_test, preds)
print(f'Model Accuracy on Test Data = {round(test_accuracy*100,2)}%')

Model Accuracy on Test Data = 55.68%


In [21]:
print('Accuracy Score Train:', logreg.score(X_train_num_scaled,y_train))
print('Accuracy Score Test:', logreg.score(X_test_num_scaled,y_test))


Accuracy Score Train: 0.557002119965083
Accuracy Score Test: 0.5567527123082678


In [22]:
log_loss_logreg = cross_val_score(logreg, X_train_num_scaled, y_train, scoring='neg_log_loss')
log_loss_logreg = -log_loss_logreg.mean()
print('Log Loss:', log_loss_logreg)

Log Loss: 0.8678721830106181


Our scores have not improved as much on our dummy model. Let's be more selective with the columns we use.

# Finding the best predictor columns

OneHotEncoding our well status

In [23]:
status_df = pd.get_dummies(df['status_group'])

status_df

,functional,functional needs repair,non functional
id,,,
3433,1,0,0
14245,1,0,0
20330,1,0,0
10321,0,1,0
74164,0,0,1
...,...,...,...
21823,1,0,0
45989,1,0,0
53307,0,0,1


'funder' and 'installer' have 1557 and 1748 unique values respectively. I would prefer not to OneHotEncode these columns. So I will drop these columns. 

In [24]:
new_df_objectonly = df.select_dtypes(include= object).drop(['funder','installer'],axis=1)
new_df_objectonly

,basin,region,public_meeting,scheme_management,permit,extraction_type,extraction_type_group,extraction_type_class,management,management_group,...,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,
3433,Ruvuma / Southern Coast,Ruvuma,True,VWC,False,other - swn 81,other handpump,handpump,vwc,user-group,...,annually,soft,good,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
14245,Lake Tanganyika,Kigoma,True,Water authority,False,other,other,other,vwc,user-group,...,never pay,soft,good,enough,spring,spring,groundwater,improved spring,improved spring,functional
20330,Lake Victoria,Kagera,True,VWC,True,afridev,afridev,handpump,vwc,user-group,...,never pay,salty,salty,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
10321,Lake Victoria,Shinyanga,True,WUG,False,nira/tanira,nira/tanira,handpump,wug,user-group,...,on failure,salty abandoned,salty,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair
74164,Lake Victoria,Shinyanga,missing,WUG,False,swn 80,swn 80,handpump,wug,user-group,...,unknown,unknown,unknown,dry,shallow well,shallow well,groundwater,hand pump,hand pump,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21823,Rufiji,Iringa,False,VWC,True,nira/tanira,nira/tanira,handpump,vwc,user-group,...,never pay,soft,good,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
45989,Lake Nyasa,Mbeya,True,VWC,False,nira/tanira,nira/tanira,handpump,vwc,user-group,...,on failure,soft,good,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
53307,Lake Tanganyika,Rukwa,False,VWC,True,ksb,submersible,submersible,vwc,user-group,...,never pay,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,non functional


Let's look at how many unique values are in each column

In [25]:
[len(new_df_objectonly[a].value_counts()) for a in new_df_objectonly.columns]

[9, 21, 3, 13, 3, 18, 13, 7, 12, 5, 7, 7, 8, 6, 5, 10, 7, 3, 7, 6, 3]

Below is a list of the column names in the same corresponding order as the list of value_counts above.

In [26]:
[a for a in new_df_objectonly.columns]

['basin',
 'region',
 'public_meeting',
 'scheme_management',
 'permit',
 'extraction_type',
 'extraction_type_group',
 'extraction_type_class',
 'management',
 'management_group',
 'payment',
 'payment_type',
 'water_quality',
 'quality_group',
 'quantity',
 'source',
 'source_type',
 'source_class',
 'waterpoint_type',
 'waterpoint_type_group',
 'status_group']

## 'source_class' as a predictor

In [27]:
source_class_df = pd.get_dummies(df['source_class'])

logreg = LogisticRegression()
ss = StandardScaler()

X_trainSource, X_testSource, y_trainSource, y_testSource = train_test_split(source_class_df, status_df['functional'], stratify=y, random_state=42, test_size= 0.2)
logreg.fit(X_trainSource, y_trainSource)

preds = logreg.predict(X_testSource)
print (f'Model Accuracy Score : {round((accuracy_score(y_testSource, preds))*100,4)}%')

log_loss_logregnew = cross_val_score(logreg, X_trainSource, y_trainSource, scoring='neg_log_loss')
log_loss_logregnew = -log_loss_logregnew.mean()
print('Log Loss:', log_loss_logregnew)

Model Accuracy Score : 54.2555%
Log Loss: 0.6895434205137672


## 'management_group' as a predictor

In [28]:
management_group_df = pd.get_dummies(df['management_group'])

logreg = LogisticRegression()
ss = StandardScaler()

X_train_MgtGr, X_test_MgtGr, y_train_MgtGr, y_test_MgtGr = train_test_split(management_group_df[['commercial','parastatal','user-group']], status_df['functional'], stratify=y, random_state=42, test_size= 0.2)

logreg.fit(X_trainSource, y_trainSource)


preds = logreg.predict(X_test_MgtGr)
print (f'Model Accuracy Score : {round((accuracy_score(y_test_MgtGr, preds))*100,4)}%')

log_loss_logreg_MgtGr = cross_val_score(logreg, X_train_MgtGr, y_train_MgtGr, scoring='neg_log_loss')
log_loss_logreg_MgtGr = -log_loss_logreg_MgtGr.mean()
print('Log Loss:', log_loss_logreg_MgtGr)

Model Accuracy Score : 54.2555%
Log Loss: 0.6889178316296157


## Iterating through all columns as predictors

We used 'source_class' and 'management_group' as a predictor. Let's just try this on every other column with dtype= object.

In [29]:
def find_predict(columns):
    
    status_df = pd.get_dummies(df['status_group'])
    
    for a in columns.columns:

        predictor = pd.get_dummies(df[a])

        logreg = LogisticRegression()
        ss = StandardScaler()

        X_trainAA, X_testAA, y_trainAA, y_testAA = train_test_split(predictor, status_df['functional'], stratify=y, random_state=42, test_size= 0.2)
        logreg.fit(X_trainAA, y_trainAA)

        preds = logreg.predict(X_testAA)
    

        log_loss_logregnew = cross_val_score(logreg, X_trainAA, y_trainAA, scoring='neg_log_loss')
        log_loss_logregnew = -log_loss_logregnew.mean()
        
        
        print (f'Column Used: {a}')
        print (f'Model Accuracy Score : {round((accuracy_score(y_testSource, preds))*100,4)}%')
        print('Log Loss:', log_loss_logregnew)
        print('-----------------------------')

In [30]:
find_predict(new_df_objectonly)

Column Used: basin
Model Accuracy Score : 57.3326%
Log Loss: 0.6760463555163326
-----------------------------
Column Used: region
Model Accuracy Score : 59.119%
Log Loss: 0.6640353864217361
-----------------------------
Column Used: public_meeting
Model Accuracy Score : 55.2189%
Log Loss: 0.6867916127208036
-----------------------------
Column Used: scheme_management
Model Accuracy Score : 54.349%
Log Loss: 0.6809815175523315
-----------------------------
Column Used: permit
Model Accuracy Score : 54.2555%
Log Loss: 0.6891249811891969
-----------------------------
Column Used: extraction_type
Model Accuracy Score : 63.5615%
Log Loss: 0.6460554969907655
-----------------------------
Column Used: extraction_type_group
Model Accuracy Score : 63.8608%
Log Loss: 0.646210821667848
-----------------------------
Column Used: extraction_type_class
Model Accuracy Score : 63.7673%
Log Loss: 0.6474256082760176
-----------------------------
Column Used: management
Model Accuracy Score : 54.5829%
Lo

## Binarizing Construction Year

In [161]:
year_df = df[df['construction_year'] != 0]
year_df

,amount_tsh,funder,gps_height,installer,longitude,latitude,num_private,basin,region,region_code,...,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
id,,,,,,,,,,,,,,,,,,,,,
3433,5000.0,Kiuma,590,KIUMA,37.185254,-11.035441,0,Ruvuma / Southern Coast,Ruvuma,10,...,annually,soft,good,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
14245,0.0,Villagers,1671,Villagers,29.773515,-4.427672,0,Lake Tanganyika,Kigoma,16,...,never pay,soft,good,enough,spring,spring,groundwater,improved spring,improved spring,functional
7884,0.0,Kiliwater,1693,Kiliwater,37.575545,-3.051765,0,Internal,Kilimanjaro,3,...,monthly,soft,good,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,non functional
33133,40.0,Private,32,Private,39.286660,-6.978073,0,Wami / Ruvu,Pwani,60,...,per bucket,soft,good,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,functional
532,0.0,District Council,681,District Council,37.532074,-3.534174,0,Pangani,Kilimanjaro,3,...,never pay,soft,good,enough,shallow well,shallow well,groundwater,hand pump,hand pump,non functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70399,2000.0,Tasaf,374,TASAF,37.364728,-11.444121,0,Ruvuma / Southern Coast,Ruvuma,10,...,annually,soft,good,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
21823,0.0,Kkkt,1681,KKKT,34.924950,-8.456492,0,Rufiji,Iringa,11,...,never pay,soft,good,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional
53307,0.0,Go,1730,DWE,31.368730,-7.972788,0,Lake Tanganyika,Rukwa,15,...,never pay,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,non functional


In [202]:
#A dataframe that does not include values where 'construction_year'= 0
year_df = df[df['construction_year'] != 0]

#binarizing the 'construction_year' column as a new column named 'built_21century'
year_df['built_21century'] =  year_df['construction_year'].apply(lambda x: 1 if x>= 2000 else 0)

#binarizing the 'status_group' column as a new column named 'functional_binary'
year_df['functional_binary'] = year_df['status_group'].apply(lambda x:1 if x== 'functional' else 0)

#using get_dummies() on the 'built_21century' column as it will be our predictor
age_dummy_df = pd.get_dummies(year_df['built_21century'])


#Applying Logistic Regression

X = age_dummy_df
y = age_function_binary_df['functional_binary']

X_trainAGE, X_testAGE, y_trainAGE, y_testAGE = train_test_split(X, y, stratify=y, random_state=42, test_size= 0.25)

logreg = LogisticRegression()

logreg.fit(X_trainAGE, y_trainAGE)

score_train =logreg.score(X_trainAGE, y_trainAGE)
score_test =logreg.score(X_testAGE, y_testAGE)

log_loss_age_train = cross_val_score(logreg, X_trainAGE, y_trainAGE, scoring='neg_log_loss')
log_loss_age_train = -log_loss_age_train.mean()

log_loss_age_test = cross_val_score(logreg, X_testAGE, y_testAGE, scoring='neg_log_loss')
log_loss_age_test = -log_loss_age_test.mean()

print(f'Training Score: {score_train}')
print(f'Testing Score: {score_test}')
print(f'Log Loss Training: {log_loss_age_train}')
print(f'Log Loss Test: {log_loss_age_test}')



<ipython-input-202-5631e9659d24>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['built_21century'] =  year_df['construction_year'].apply(lambda x: 1 if x>= 2000 else 0)
<ipython-input-202-5631e9659d24>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  year_df['functional_binary'] = year_df['status_group'].apply(lambda x:1 if x== 'functional' else 0)


Training Score: 0.6212956581667816
Testing Score: 0.6237510049385552
Log Loss Training: 0.6576568373417891
Log Loss Test: 0.6563387702523158
